<a href="https://colab.research.google.com/github/RealMyeong/Aiffel_Exploration/blob/main/EX_BS2_project13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [1]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.6
1.7.3
0.6.1


1) 데이터 준비와 전처리
---
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [3]:
import pandas as pd
rating_file_path='/content/drive/MyDrive/AIFFEL/EX/13.좋아하는 아티스트 찾기/project/data_movie/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 3점 이상만 남깁니다.
ratings_2 = ratings.copy()
ratings_2 = ratings_2[ratings_2['ratings']>=3]
filtered_data_size = len(ratings_2)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
ratings_2.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
ratings_2.rename(columns={'ratings':'counts'}, inplace=True)

In [7]:
ratings_2['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [8]:
ratings_2.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


timestamp는 필요 없을 것 같다.

In [9]:
use_columns = ['user_id', 'movie_id', 'counts']
ratings_2 = ratings_2[use_columns]
ratings_2.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [10]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path='/content/drive/MyDrive/AIFFEL/EX/13.좋아하는 아티스트 찾기/project/data_movie/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


영화 제목과 movie_id를 한 번에 보기 위해 데이터 합침

In [11]:
merge_data = pd.merge(ratings_2, movies, on='movie_id')

In [12]:
merge_data.tail()

,user_id,movie_id,counts,title,genre
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,Identification of a Woman (Identificazione di ...,Drama


In [13]:
all_data = merge_data.copy()

In [14]:
all_data.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama


In [15]:
all_data.tail()

,user_id,movie_id,counts,title,genre
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,Identification of a Woman (Identificazione di ...,Drama


첫 번째 유저가 어떤 영화를 봤는지 확인

In [16]:
condition = (all_data['user_id']== all_data.loc[0, 'user_id'])
all_data.loc[condition]

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1680,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2123,1,914,3,My Fair Lady (1964),Musical|Romance
2734,1,3408,4,Erin Brockovich (2000),Drama
3957,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
5556,1,1197,3,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
7808,1,1287,5,Ben-Hur (1959),Action|Adventure|Drama
8474,1,2804,5,"Christmas Story, A (1983)",Comedy|Drama
9764,1,594,4,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
10471,1,919,4,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical


## 데이터 분석

데이터 탐색
---
추천 모델을 만들기 전에 데이터의 기본적인 정보를 보고 갑시다. 아래 항목들을 확인해 보고 싶습니다.

- 유저 수, 영화 수, 인기 많은 영화
- 유저들이 몇 개의 영화를 보고 있는지에 대한 통계  
(참고) pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용합니다.

In [17]:
# 유저 수
a = all_data['user_id'].nunique()
# 영화 개수
b = all_data['title'].nunique()

print(f'총 유저 수 : {a}')
print(f'총 영화 개수 : {b}')

총 유저 수 : 6039
총 영화 개수 : 3628


In [18]:
all_data['user_id'].max()

6040

In [19]:
# 가장 인기가 많은 영화
m_count = all_data.groupby('title')['user_id'].count()
m_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

가장 인기있는 영화 30개를 뽑아보면 실제로 보지는 못했어도 이름은 들어본 근본 영화들이 많다.

In [20]:
# 유저 별 영화를 몇개정도 보는지
user_count = all_data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

각 유저마다 영화를 얼마나 보는지에 대해 봤는데 1968개를 본 사람은...? 

## 선호하는 영화 5개 추가해보기

내가 마음에 드는 영화로 5개를 골라 기존의 데이터에 넣어준다.

In [21]:
# 영화 제목으로 장르 추출하는 함수
def get_genre_of_movie(my_favorite):
    genres_of_my_favorite = []
    for k, v in my_favorite.items():
        genres_of_my_favorite.extend(movies[movies['title'] == v]['genre'].to_list())
    return genres_of_my_favorite

In [22]:
# 영화 데이터 추가! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요.
my_favorite = {
    70: "From Dusk Till Dawn (1996)",
    164: "Devil in a Blue Dress (1995)",
    293: "Professional, The (a.k.a. Leon: The Professional) (1994)",
    426: "Body Snatchers (1993)",
    3476: "Jacob's Ladder (1990)",
}

# 6041이라는 user_id가 위 영화를 counts회씩 시청했다고 가정하겠습니다.
my_playlist = pd.DataFrame({
    'user_id': [6041]*5,
    'movie_id': my_favorite.keys(),
    'counts': [3,4,4,5,5],
    'title': my_favorite.values(),
    'genre': get_genre_of_movie(my_favorite)
})

if not all_data.isin({'user_id':[6041]})['user_id'].any():  # user_id에 6041이라는 데이터가 없다면
    all_data = all_data.append(my_playlist)  # my_favorite 데이터 추가

all_data.tail(10)  # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title,genre
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,Identification of a Woman (Identificazione di ...,Drama
0,6041,70,3,From Dusk Till Dawn (1996),Action|Comedy|Crime|Horror|Thriller
1,6041,164,4,Devil in a Blue Dress (1995),Crime|Film-Noir|Mystery|Thriller
2,6041,293,4,"Professional, The (a.k.a. Leon: The Profession...",Crime|Drama|Romance|Thriller
3,6041,426,5,Body Snatchers (1993),Horror|Sci-Fi|Thriller
4,6041,3476,5,Jacob's Ladder (1990),Horror|Mystery|Thriller


- 한 영화에도 여러개의 genre가 섞여있는 영화가 있기 때문에 장르의 종류가 얼마나 되는지 리스트와 Counter를 이용해서 계산해줌

In [ ]:
from collections import Counter

genre_list = []
for i in all_data['genre']:
  for j in range(len(i)):
    genre_list.append(i[j])

genre_count = Counter(genre_list)
genre_count

Counter({'Drama': 313263,
         'Animation': 37610,
         "Children's": 57676,
         'Musical': 35661,
         'Romance': 125928,
         'Comedy': 293893,
         'Action': 209496,
         'Adventure': 108820,
         'Fantasy': 29206,
         'Sci-Fi': 126091,
         'War': 61226,
         'Thriller': 158344,
         'Crime': 68890,
         'Mystery': 34417,
         'Western': 17668,
         'Horror': 55488,
         'Film-Noir': 17166,
         'Documentary': 7162})

In [ ]:
genre_count.keys()

dict_keys(['Drama', 'Animation', "Children's", 'Musical', 'Romance', 'Comedy', 'Action', 'Adventure', 'Fantasy', 'Sci-Fi', 'War', 'Thriller', 'Crime', 'Mystery', 'Western', 'Horror', 'Film-Noir', 'Documentary'])

총 장르의 종류는 18종류이다. 여러개의 장르가 섞여 있는 영화의 장르를 어떻게 표현 할까 생각하다가 처음에는 one-hot encoding으로 해볼까 라는 생각을 했지만 데이터가 너무 커질 것 같아서 애초에 영화를 만들 때 genre가 섞여서 만들어지기 때문에 각 장르마다 index를 부여

In [ ]:
genre_index = {'Drama': 1,
         'Animation': 2,
         "Children's": 3,
         'Musical': 4,
         'Romance': 5,
         'Comedy': 6,
         'Action': 7,
         'Adventure': 8,
         'Fantasy': 9,
         'Sci-Fi': 10,
         'War': 11,
         'Thriller': 12,
         'Crime': 13,
         'Mystery': 14,
         'Western': 15,
         'Horror': 16,
         'Film-Noir': 17,
         'Documentary': 18}

# 각 장르를 정해둔 인덱스값으로 바꿔줌
for i in all_data['genre']:
  for j in range(len(i)):
    i[j] = genre_index[i[j]]
  

In [ ]:
all_data.tail(20)

,user_id,movie_id,counts,title,genre
836463,5328,2438,4,Outside Ozona (1998),"[1, 144]"
836464,5334,3323,3,Chain of Fools (2000),"[36, 169]"
836465,5334,3382,5,Song of Freedom (1936),[1]
836466,5420,1843,3,Slappy and the Stinkers (1998),"[9, 36]"
836467,5433,286,3,Nemesis 2: Nebula (1995),"[49, 100, 144]"
836468,5494,3530,4,Smoking/No Smoking (1993),[36]
836469,5556,2198,3,Modulations (1998),[324]
836470,5949,2198,5,Modulations (1998),[324]
836471,5675,2703,3,Broken Vessels (1998),[1]
836472,5717,2258,4,Master Ninja I (1984),[49]


In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836483 entries, 0 to 4
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   836483 non-null  int64  
 1   movie_id  836483 non-null  int64  
 2   counts    836483 non-null  int64  
 3   title     836483 non-null  object 
 4   genre     836483 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 38.3+ MB


각 장르가 하나의 숫자로 잘 바뀐것을 볼 수 있다.


모델에서 활용하기 위해 인덱싱 과정 

In [23]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = all_data['user_id'].unique()
movie_unique = all_data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [24]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print('user_id indexing: {}\n'.format(user_to_idx[6041]))

for k, v in my_favorite.items():
    print('movie_id indexing: {}'.format(movie_to_idx[v]))

user_id indexing: 6039

movie_id indexing: 1034
movie_id indexing: 864
movie_id indexing: 890
movie_id indexing: 2361
movie_id indexing: 279


In [25]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_all_data = all_data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_all_data) == len(all_data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    all_data['user_id'] = temp_user_all_data   # all_data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
# temp_movie_all_data = all_data['movie_id'].map(movie_to_idx.get).dropna()
temp_movie_all_data = all_data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_all_data) == len(all_data):
    print('movie column indexing OK!!')
    all_data['movie_id'] = temp_movie_all_data
else:
    print('movie column indexing Fail!!')

all_data.tail(10)

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,title,genre
836473,1621,3623,5,One Little Indian (1973),Comedy|Drama|Western
836474,3481,3624,4,Slaughterhouse (1987),Horror
836475,3481,3625,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,4159,3626,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,1648,3627,5,Identification of a Woman (Identificazione di ...,Drama
0,6039,1034,3,From Dusk Till Dawn (1996),Action|Comedy|Crime|Horror|Thriller
1,6039,864,4,Devil in a Blue Dress (1995),Crime|Film-Noir|Mystery|Thriller
2,6039,890,4,"Professional, The (a.k.a. Leon: The Profession...",Crime|Drama|Romance|Thriller
3,6039,2361,5,Body Snatchers (1993),Horror|Sci-Fi|Thriller
4,6039,279,5,Jacob's Ladder (1990),Horror|Mystery|Thriller


In [26]:
all_data.tail(10)

,user_id,movie_id,counts,title,genre
836473,1621,3623,5,One Little Indian (1973),Comedy|Drama|Western
836474,3481,3624,4,Slaughterhouse (1987),Horror
836475,3481,3625,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,4159,3626,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,1648,3627,5,Identification of a Woman (Identificazione di ...,Drama
0,6039,1034,3,From Dusk Till Dawn (1996),Action|Comedy|Crime|Horror|Thriller
1,6039,864,4,Devil in a Blue Dress (1995),Crime|Film-Noir|Mystery|Thriller
2,6039,890,4,"Professional, The (a.k.a. Leon: The Profession...",Crime|Drama|Romance|Thriller
3,6039,2361,5,Body Snatchers (1993),Horror|Sci-Fi|Thriller
4,6039,279,5,Jacob's Ladder (1990),Horror|Mystery|Thriller


In [27]:
all_data['movie_id'].nunique(), all_data['genre'].nunique()

(3628, 301)

~~영화의 제목으로 추천을 해주는 것 보다는 영화의 장르에 따라서 추천을 해주는 게 좋을 것 같음.
따라서 movie_id는 사용하지않고 genre만 사용~~

genre로 사용하려고 했는데 데이터가 너무 줄어버림 -> 아마 같은 장르지만 제목만 다른 영화들이 있어서 그런 것 같다. 크게 의미 없는 행동 이었던 걸로... ㅎㅎ

In [28]:
all_data.drop(['genre'], axis=1, inplace=True)

In [29]:
all_data.tail(10)

,user_id,movie_id,counts,title
836473,1621,3623,5,One Little Indian (1973)
836474,3481,3624,4,Slaughterhouse (1987)
836475,3481,3625,3,"Promise, The (Versprechen, Das) (1994)"
836476,4159,3626,4,"Five Wives, Three Secretaries and Me (1998)"
836477,1648,3627,5,Identification of a Woman (Identificazione di ...
0,6039,1034,3,From Dusk Till Dawn (1996)
1,6039,864,4,Devil in a Blue Dress (1995)
2,6039,890,4,"Professional, The (a.k.a. Leon: The Profession..."
3,6039,2361,5,Body Snatchers (1993)
4,6039,279,5,Jacob's Ladder (1990)


In [30]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836483 entries, 0 to 4
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   836483 non-null  int64 
 1   movie_id  836483 non-null  int64 
 2   counts    836483 non-null  int64 
 3   title     836483 non-null  object
dtypes: int64(3), object(1)
memory usage: 31.9+ MB


In [31]:
all_data.isnull().sum()

user_id     0
movie_id    0
counts      0
title       0
dtype: int64

## CSR matrix 생성

In [32]:
from scipy.sparse import csr_matrix

num_user = all_data['user_id'].nunique()
num_movie = all_data['movie_id'].nunique()

csr_data = csr_matrix((all_data.counts, (all_data.user_id, all_data.movie_id)), shape = (num_user, num_movie))

In [33]:
csr_data.shape

(6040, 3628)

## MF 모델 학습하기

In [34]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다.

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지  

1과 4를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [36]:
# implicit alternatingLeastSquares 모델 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [37]:
#als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [38]:
#모델 훈련
als_model.fit(csr_data)

  0%|          | 0/20 [00:00<?, ?it/s]

## 영화의 선호도 예측

In [39]:
user = user_to_idx[6041]
user_vector = als_model.user_factors[user]
user_vector

array([ 0.7310223 ,  0.16999468,  0.3972833 ,  0.05521971,  0.7740921 ,
       -0.36071777,  0.4227172 , -0.11384459, -0.04317628, -0.345832  ,
        0.08061555,  0.30303517, -0.22123714,  0.01144469, -0.23566087,
        0.74462324, -0.17289278, -0.11890782,  0.0371611 ,  0.04656414,
       -0.11134008, -0.41757077,  0.20390372,  0.30380026,  0.21505666,
       -0.01920786,  0.52788776,  0.32721356, -0.28389436, -0.9545049 ,
        0.5544715 ,  0.1298983 , -0.05484161, -0.250074  ,  0.09057002,
       -0.3484159 , -0.3282362 , -0.21428935,  0.67455536, -0.31327063,
       -0.11660857, -0.14718075, -0.69953537, -0.16836177, -0.25447828,
       -0.5033315 , -0.30714777,  0.42514086, -0.12920967, -0.26635373,
        0.3240679 ,  0.9813298 ,  0.19304164,  0.14731812, -0.448488  ,
        0.39054355,  0.06698325, -0.23859897, -0.44227806,  0.08240768,
       -0.2337878 , -0.21044327, -0.22483937,  0.17792569, -0.21591593,
        0.06080162,  0.32062066,  0.1946992 , -0.15713911,  0.20

In [40]:
star_wars_epi_4 = movie_to_idx['Star Wars: Episode IV - A New Hope (1977)']
star_wars_epi_4_vector = als_model.item_factors[star_wars_epi_4]
star_wars_epi_4_vector  # 영화 벡터

array([-8.71433318e-03,  2.27086879e-02, -4.64785518e-03,  1.93228312e-02,
        1.73320919e-02,  2.49706004e-02,  8.00065603e-03, -2.73421314e-02,
        1.33530125e-02,  1.38007086e-02,  1.91657413e-02,  2.54061632e-03,
        6.43981714e-03,  2.12699249e-02, -2.75433380e-02, -1.43816520e-03,
        6.80824229e-03,  8.98958370e-03,  1.00832833e-02,  3.43371881e-03,
       -6.15384197e-03,  1.11416075e-02,  2.71786600e-02, -1.49334176e-02,
       -6.45807813e-05, -1.52496574e-03,  3.41095291e-02, -7.52999727e-03,
       -3.34821176e-03, -3.50065976e-02,  3.26622352e-02,  1.58132166e-02,
        1.97577272e-02,  1.00099053e-02,  7.21494760e-03,  1.24592800e-02,
        2.38119392e-03,  1.12624625e-02, -2.46008113e-02, -5.78925526e-03,
        3.50351888e-03,  2.59101135e-03,  1.06166070e-02,  3.14151607e-02,
       -3.41501553e-04,  1.26270775e-03, -1.65454354e-02,  1.05378758e-02,
        2.57559866e-02,  2.02592388e-02,  2.57834718e-02, -2.15786770e-02,
        1.98276713e-02, -

In [43]:
dusk_till_dawn = movie_to_idx["From Dusk Till Dawn (1996)"]
dusk_till_dawn_vector = als_model.item_factors[dusk_till_dawn]
dusk_till_dawn_vector

array([ 1.25188017e-02,  9.75797605e-03,  1.80342011e-02, -7.79028656e-03,
        4.00695298e-03, -5.15485555e-03,  9.24235303e-03, -1.71732865e-02,
        8.47957749e-03,  9.91808809e-03,  2.33273245e-02,  7.84699526e-03,
        5.31863887e-03,  1.69174070e-03,  1.49681186e-02,  1.45457154e-02,
        7.00568082e-03, -6.29975228e-03, -1.31499367e-02,  2.67341291e-03,
       -1.15685165e-02, -4.92107822e-03,  2.60463916e-02,  2.48611495e-02,
        7.28129968e-03, -1.91064249e-03,  9.06703249e-03,  4.84225675e-02,
        1.30173576e-03, -5.67231211e-04,  4.89694066e-03,  2.10707914e-02,
        1.33307502e-02, -7.52546731e-03,  4.47471580e-03, -4.27521300e-03,
       -1.00251622e-02, -1.23444395e-02,  1.02088423e-02,  8.19692854e-03,
       -7.18025840e-04,  7.83291087e-03, -2.12024618e-02,  7.01607624e-03,
        1.03437454e-02,  1.29443565e-02,  1.11971190e-02,  3.25288884e-02,
        1.11520765e-02, -1.42325275e-02,  2.28241030e-02,  2.37542000e-02,
        1.16556836e-02, -

- 싫어하는 영화와의 예측 선호도

In [42]:
np.dot(user_vector, star_wars_epi_4_vector)

-0.012408019

- 좋아하는 영화와의 예측 선호도


In [44]:
np.dot(user_vector, dusk_till_dawn_vector)

0.34838888

좋아한다고 넣은 영환데 0.34밖에 나오지 않음...

## 좋아하는 영화와 비슷한 영화 추천받기

In [45]:
favorite_movie = "From Dusk Till Dawn (1996)"
movie_title = movie_to_idx[favorite_movie]
similar_movies = als_model.similar_items(movie_title, N=15)
similar_movies


(array([1034, 2617, 1697, 1748, 1794,  999, 1303, 1008, 2610, 1779, 2253,
        1631,  495, 1824, 2484], dtype=int32),
 array([1.        , 0.45006862, 0.43013874, 0.42205358, 0.42044252,
        0.42003885, 0.41130948, 0.41101453, 0.40430945, 0.40382993,
        0.4031765 , 0.40157816, 0.40092614, 0.39859673, 0.3968744 ],
       dtype=float32))

In [48]:
similar_movies[0]

array([1034, 2617, 1697, 1748, 1794,  999, 1303, 1008, 2610, 1779, 2253,
       1631,  495, 1824, 2484], dtype=int32)

In [49]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i] for i in similar_movies[0]]

['From Dusk Till Dawn (1996)',
 'Tales from the Crypt Presents: Bordello of Blood (1996)',
 'Vampires (1998)',
 'Prophecy, The (1995)',
 'Hellbound: Hellraiser II (1988)',
 'Evil Dead II (Dead By Dawn) (1987)',
 'King of New York (1990)',
 'Tales from the Hood (1995)',
 'Candyman (1992)',
 'American Werewolf in Paris, An (1997)',
 'City of Industry (1997)',
 'Deep Rising (1998)',
 'Prophecy II, The (1998)',
 'Otello (1986)',
 'Beyond the Mat (2000)']

In [50]:
def get_similar_movie(movie_name: str):
    movie_title = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_title, N=15)
    similar_movie = [idx_to_movie[i] for i in similar_movies[0]]
    return similar_movie

In [51]:
get_similar_movie('Deep Impact (1998)')

['From Dusk Till Dawn (1996)',
 'Tales from the Crypt Presents: Bordello of Blood (1996)',
 'Vampires (1998)',
 'Prophecy, The (1995)',
 'Hellbound: Hellraiser II (1988)',
 'Evil Dead II (Dead By Dawn) (1987)',
 'King of New York (1990)',
 'Tales from the Hood (1995)',
 'Candyman (1992)',
 'American Werewolf in Paris, An (1997)',
 'City of Industry (1997)',
 'Deep Rising (1998)',
 'Prophecy II, The (1998)',
 'Otello (1986)',
 'Beyond the Mat (2000)']

## 내가 좋아할만한 영화 추천받기

In [53]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data[user], N=20, filter_already_liked_items=True)
movie_recommended

(array([ 299,  831,  997,  440,  855,  919, 1090,  317, 1107,  272,  496,
         210,  289, 1486,  295, 1078, 1113, 1368,   72,  870], dtype=int32),
 array([0.23370406, 0.2130478 , 0.21101744, 0.20986609, 0.20824628,
        0.20080692, 0.20028043, 0.18374425, 0.17125106, 0.16606128,
        0.16472429, 0.16264856, 0.15973511, 0.15442939, 0.15423895,
        0.15326825, 0.15290879, 0.14278011, 0.13955615, 0.13866575],
       dtype=float32))

추천 영화와 점수 확인

In [60]:
recommended = [idx_to_movie[i] for i in movie_recommended[0]]
score = [i for i in movie_recommended[1]]
pd.DataFrame({'movie': recommended, 'score' : score})

,movie,score
0,"Mariachi, El (1992)",0.233704
1,"Devil's Advocate, The (1997)",0.213048
2,Scream (1996),0.211017
3,True Romance (1993),0.209866
4,"Grifters, The (1990)",0.208246
5,Mulholland Falls (1996),0.200807
6,Nikita (La Femme Nikita) (1990),0.200280
7,Twelve Monkeys (1995),0.183744
8,Rosemary's Baby (1968),0.171251
9,Grosse Pointe Blank (1997),0.166061


In [56]:
item_movie = movie_to_idx['Kansas City (1996)']
explain = als_model.explain(user, csr_data, itemid=item_movie)

In [58]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('From Dusk Till Dawn (1996)', 0.011499829995023143),
 ('Body Snatchers (1993)', 0.009092058514848306),
 ("Jacob's Ladder (1990)", 0.0011763307529632744),
 ('Devil in a Blue Dress (1995)', -0.002291428347396875),
 ('Professional, The (a.k.a. Leon: The Professional) (1994)',
  -0.003726907934154824)]